# Исследование надежности заемщиков.

## Описание проекта

В проекте рассматривается гипотеза о взаимосвязи семейного положения и количества детей заемщика на факт погашения кредита в срок. Исследование проходит путем изучения датафрейма с данными клиентов, в процессе исследования изучаются дубликаты и аномалии данных, происходит обработка пропусков и затем при помощи подготовленных данных происходит изучение поставленной гипотезы. Нахождение зависимостей между параметрами проводится с поощью сводных таблиц. Проект направлен на изучение темы предобработки и категоризации данных.

## Обзор данных

Для начала исследования испортируем библиотеку `pandas`, присвоим ей псевдоним pd и откроем датасет при помощи команды `read_csv`. Далее при помощи метода `info()` исследуем основную информацию о датафрейме, такую как: количество и названия столбцов, количество записей и тип данных каждого столбца.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Как видно, в нашем датафрейме есть 12 столбцов (5 из них- с целочисленным типом `int64`, 5 - с текстовым типом `object` и 2 - с типом с плавающей точкой `float64`). Названия столбцов корректные, соответствуют стилю и отображают смысл столбцов. В двух столбцах (**days_employed, total_income**) есть пропуски значений (19351 значений против 21525 во всех остальных столбцах). Необходимо исследовать эти пропуски, изучить их характер и заполнить недостающие значения для дальнейшей работы с датафреймом. 

## Предобработка данных

### Заполнение пропусков

Для исследования пропусков воспользуемся методом `isna()`, функцией суммы `sum()` и посчитаем, сколько пропусков есть в столбцах **days_employed** и **total_income**: 

In [2]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Как видно, в каждом из этих столбцов есть 2174 пропуска. Посмотрим визуально, выгрузив первые 15 строк датафрейма, какого характера эти пропуски и если они ожидаемые (в данному случае `NaN`, т.к. эти столбцы имеют тип `float64` (т.е. число с плавающей точкой), тогда воспользуемся методом заполнения ожидаемых пропусков `fillna()`

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Визуально в 12 строке определяются ожидаемые пропуски- `NaN`. Подобные пропуски могут быть связаны с небрежностью при заполнении данных, т.к. данные о трудовом стаже и ежемесячном доходе клиента являются фундаментальными данными, на которых базируется любое скоринговое исследование о платежоспособности клиента в принципе. Т.к. пропуски в столбцах **days_employed** и **total_income** недопустимы, а удаление этих строк может привести к некорректному результату исследования (процент пропущенных данных в этих столбцах равен ок. 10%, что довольно много), используем метод `fillna()` и заполним пропущенные данные медианными значениями стажа и заработка при помощи функции `median()`. 

Здесь мы будем применять именно медианное значение, т.к. именно медиана используется для количественных переменных, которыми и являются значения в обоих столбцах. 

Найдем медианные значения для стобцов **days_employed** и **total_income**: 

In [4]:
median_days_employed = data['days_employed'].median()
median_total_income = data['total_income'].median()

print(median_days_employed)
median_total_income

-1203.369528770489


145017.93753253992

Мы получили медианные значения в столбцах **days_employed** и **total_income** и видим, что медианное значение в столбце **days_employed** - некорректно (трудовой стаж не может иметь отрицательных значений). Возможно, эта ошибка возникла при переносе данных или при заполнении исходных форм. 
Перед нахождением медианного значения будет правильно сменить знак `-` в столбце **days_employed** на `+` и только затем найти медианное значение. Используем для замены значений логическую индексацию и функцию нахождения абсолютного значения `abs()`, а затем выведем датафрейм на экран и отсортируем по сначала по убыванию столбца **days_employed**, а затем по возрастанию: 

In [5]:
data.loc[data['days_employed'] < 0, 'days_employed'] = data['days_employed'].abs()

display(data.sort_values(by = 'days_employed', ascending = False).head(10))

data.sort_values(by = 'days_employed').head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
4697,0,401635.032697,56,среднее,1,женат / замужем,0,F,пенсионер,0,48242.322502,покупка недвижимости
13420,0,401619.633298,63,Среднее,1,гражданский брак,1,F,пенсионер,0,51449.788325,сыграть свадьбу
17823,0,401614.475622,59,среднее,1,женат / замужем,0,F,пенсионер,0,152769.694536,покупка жилья для сдачи
10991,0,401591.828457,56,среднее,1,в разводе,3,F,пенсионер,0,39513.517543,получение дополнительного образования
8369,0,401590.452231,58,среднее,1,женат / замужем,0,F,пенсионер,0,175306.312902,образование


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
17437,1,24.141633,31,среднее,1,женат / замужем,0,F,сотрудник,1,166952.415427,высшее образование
8336,0,24.240695,32,высшее,0,Не женат / не замужем,4,M,сотрудник,0,124115.373655,получение дополнительного образования
6157,2,30.195337,47,среднее,1,гражданский брак,1,M,компаньон,0,231461.185606,свадьба
9683,0,33.520665,43,среднее,1,Не женат / не замужем,4,M,сотрудник,1,128555.897209,приобретение автомобиля
2127,1,34.701045,31,высшее,0,женат / замужем,0,F,компаньон,0,90557.994311,получение образования
5287,1,37.726602,36,среднее,1,женат / замужем,0,F,сотрудник,0,64954.565099,операции со своей недвижимостью
17270,0,39.954170,34,высшее,0,женат / замужем,0,F,госслужащий,0,97264.767002,жилье
13846,2,46.952793,33,среднее,1,женат / замужем,0,F,сотрудник,1,177554.195351,покупка жилья для сдачи
7964,0,47.109840,49,высшее,0,женат / замужем,0,F,сотрудник,0,197545.271278,на покупку подержанного автомобиля
3235,0,50.128298,43,среднее,1,гражданский брак,1,F,компаньон,0,99381.947946,на проведение свадьбы


Как мы видим, отрицательные значения в столбце **days_employed** теперь отсутствуют, но если посмотреть на первый вывод датафрейма- в нем присутствуют очень большие значения трудового стажа, например 401755.400475 дней, что соответствует:

In [6]:
401755.400475 / 365

1100.699727328767

1100 лет непрерывной работы! Скорей всего, здесь также закралась ошибка при переносе данных. Проверим, сколько в датафрейме строк со стажем, более 60 лет:

In [7]:
data.loc[data['days_employed'] > 50*365]['days_employed'].count()

3446

Всего в датафрейме 3446 строк со значением трудового стажа, превышающий 60 лет. На будущее для исследования необходимо выяснить у ответственных за заполнение данной таблицы лиц, не было ли допущено ошибок при заполнении, либо не было ли конвертации абсолютных дней стажа в относительные (например, увеличение при помощи каких-ли коэффициентов - северных коэффициент, вредное производство и проч). Т.к. в пояснительной записке не было указано об этом, будем считать, что в данные закралась ошибка и заменим все значения стажа более чем 50 лет на стаж, равный 50 годам: 

Заменим стаж на корректный и найдем медианное значение стажа. Т.к. бы заменяем крайне большие значения стажа просто большими, распределение не изменится и медиана должна остаться прежней. 
Найдем **median_days_employed_old** (до замены данных) и **median_days_employed_new** (после замены данных).

Проверим нашу гипотезу:

In [8]:
data_new = data
data_new.loc[data_new['days_employed'] > 50*365, 'days_employed'] = 50*365
data_new.sort_values(by = 'days_employed', ascending = False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10714,0,18250.0,63,высшее,0,вдовец / вдова,2,F,пенсионер,0,112734.356402,покупка жилья для семьи
10371,0,18250.0,62,среднее,1,женат / замужем,0,F,пенсионер,0,233129.235549,на покупку подержанного автомобиля
2777,0,18250.0,53,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,42151.122604,заняться образованием
10361,0,18250.0,59,среднее,1,женат / замужем,0,F,пенсионер,0,163220.704907,высшее образование
10348,0,18250.0,57,среднее,1,женат / замужем,0,F,пенсионер,1,109459.250272,покупка жилья


In [9]:
median_days_employed_old = data['days_employed'].median()
median_days_employed_new = data_new['days_employed'].median()

print(median_days_employed_old)
median_days_employed_new

2194.220566878695


2194.220566878695

Как мы видим, медианное значение при замене неверных значений стажа не поменялось, а максимальные значения стажа были ограничены пределом 50 лет (18250 дней). 
Таким образом, мы можем обновить наш исходный датафрейм, заменив его на измененный **data_new**:

In [10]:
data = data_new

После всех преобразований мы можем найти корректное медианное значение **median_days_employed**, и с помощью метода `fillna()` заполнить пропуски в столбцах **days_employed** и **total_income**	медианными значениями:

In [11]:
median_days_employed = data['days_employed'].median()
print(median_days_employed)

2194.220566878695


In [12]:
data['days_employed'] = data['days_employed'].fillna(value = median_days_employed)
data['total_income'] = data['total_income'].fillna(value = median_total_income)

data.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,18250.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Как мы видим, данные в строке с индексом 12, в которой присутствовали пропуски в столбцах **days_employed** и **total_income** - заполнены медианными значениями данных столбцов. Проверим при помощи `info()`, остались ли еще необработанные пропуски:

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видим, что все пропуски в столбцах заполнены. Таким образом мы можем продолжить предобработку таблицы и исследовать другие столбцы на предмет корректности данных. 

### Проверка данных на аномалии и исправления.

Начнем проверять наши данные со столбца **children** и посчитаем количество уникальных значений в столбце при помощи `value_counts()`:

In [14]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Два значения в данной таблице не соответствуют реальности: это -1 и 20. Я считаю, что обе эти ошибки возможно отнести к опечаткам и для проведения исследования возможно заменить значение -1 на 1, а 20 либо заменить на 2, либо оставить без изменения. Но т.к. суммарно количество строк с параметрами -1 и 20 в столбце **children** занимают около 0,6% от всего объема исследования, предлагаю удалить эти строки как некорректные. 

In [15]:
data_new = data.loc[(data['children'] != 20) & (data['children'] != -1)]
data_new['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Как видно, некорректные значения в столбце children удалены, и мы можем заменить наш исходный датафрейм **data** на измененный **data_new**: 

In [16]:
data = data_new

Столбец **days_employed** был досконально изучен в шаге 2.1, поэтому перейдем сразу к следующему, **dob_years**.
Посмотрим: 
1) Есть ли отрицательные значения?
2) Есть ли дети (от 0 до 18 лет)?
3) Есть ли заемщики с возрастом более 100 лет?  

In [17]:
print(data.loc[data['dob_years'] < 0]['dob_years'].count())
print(data.loc[data['dob_years'] < 18]['dob_years'].count())
print(data.loc[data['dob_years'] > 100]['dob_years'].count())

0
100
0


Видно, что отрицательных значений возраста, а также заемщиков с возрастом более 100 лет не наблюдается, зато присутствуют дети. 
Зададим фильтр на возраст менее 18 лет и посмотрим на другие значения в таблице:

In [18]:
data.loc[data['dob_years'] < 18].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,18250.000000,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,18250.000000,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


Другие параметры (например семейный статус, тип занятости и наличие дохода) указывают на опечатку при указании возраста. Т.к возраст - это количественная переменная, заполним те строки, где возраст менее 18 лет, медианным значением возраста:

In [19]:
median_dob_years = data['dob_years'].median()
print(median_dob_years)

42.0


In [20]:
data.loc[data['dob_years'] < 18,'dob_years'] = median_dob_years
data.loc[data['dob_years'] < 18,'dob_years'].count()

0

Как видим, в таблице больше нет значений возраста менее 18 лет, они заменены на медианное значение- 42 года. 

Теперь проверим столбец **education** на уникальные значения методом `value_counts()`. Видно, что в столбце присутствуют дубликаты значений, возникшие из-за написания данных в различном регистре. Отметитм этот столбец и удалим эти дубликаты в следующем шаге. 

In [21]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Проверим столбцы **education_id, family_status, family_status_id.**
Как видно из таблиц ниже, здесь отсутствуют дубликаты и все значения занесены корректно. 

In [22]:
data['education_id'].value_counts()

1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64

In [23]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [24]:
data['family_status_id'].value_counts()

0    12302
1     4160
4     2799
3     1189
2      952
Name: family_status_id, dtype: int64

Изучим уникальные значения в столбце **gender** при помощи `value_counts()`.
Видно, что у одного заемщика в поле гендера указан неявный гендер. Это можно списать либо на ошибку заполнения, либо очевидное нежелание заемщиком указывать свой гендер. Т.к. все остальные строки заполнены корректно, заменим гендер на медианное значение, а именно- на женский пол (соотношение женщин и мужчин в исследовании примерно 2:1). Затем вновь проверим заполнение при помощи метода `value_counts()` и убедимся, что нужный гендер проставлен. 

In [25]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

In [26]:
data.loc[data['gender'] =='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [27]:
data.loc[data['gender'] =='XNA','gender'] = 'F'

In [28]:
data['gender'].value_counts()

F    14155
M     7247
Name: gender, dtype: int64

Проверим оставшиеся столбцы **income_type, debt** и **total_income** на корректность значений. В столбцах **income_type** и **debt** используем метод `value_counts()` и убедимся, что некорректные значения отсутствуют. В столбце **total_income** проверим отсутствие отрицательных значений при помощи логической индексации - проверка показала, что таких значений нет, можно работать дальше. 

In [29]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [30]:
data['debt'].value_counts()

0    19670
1     1732
Name: debt, dtype: int64

In [31]:
print(data.loc[data['total_income'] < 0]['total_income'].count())

0


Последний столбец **purpose** будем проверять уже в следующих шагах, на этапе поиска дубликатов, т.к. ранее мы уже проверили, что все данные принадлежат к тестовому типу `object`, что удовлетворяет нашим условиям задачи. 

### Шаг 2.3. Изменение типов данных.

Снова запустим метод `info()` и проверим типы данных нашего датафрейма. Мы уже делали это ранее, но т.к. с датафреймом проводились преобразования, стоит получить актуальную информацию. 

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     21402 non-null  float64
 2   dob_years         21402 non-null  float64
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      21402 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 2.1+ MB


Видим, что к двум столбцам с типом `float64` (**days_employed** и **total_income**) добавился третий - **dob_years**, который ранее имел целочисленное значение (`int64`). Изменение типа связано с тем, что мы заполняли пропуски в столбце **dob_years** медианным начением возраста, которое было дано в типе `float64`. 

При изучении столбца **days_employed** видим, что использование здесь типа данных с плавающей точкой `float64` - некорректно, ведь количество дней стажа- всегда целочисленное значение. Значение среднего заработка заемщиков также удобно исследовать в целочисленном виде, изменение типа на `int` не даст нам погрешности в измерении. 

Приведем данные во всех трех столбцах к типу `int` при помощи `astype()`:

In [33]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Видно, что все численные данные датафрейма приведены к типу `int64`, текстовые остались в типе `object`. Преобразование типов завершено. 

### Удаление дубликатов.

Для корректного исследования необходимо удалить все дубликаты. Для этого распечатаем первые 5 строк таблицы и посмотрим, в каких столбцах они могут встречаться: 

In [34]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,18250,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Дубликаты могут быть в категориальных переменных, к которым у нас относятся столбцы: **education, family_status, income_type, purpose**. Проверим эти столбцы при помощи `value_counts()` и оптимизируем данные в них. Начнем со столбца **education**:

In [35]:
data['education'].value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видно, что присутствуют дубликаты, связанные из-за ввода данных в различном регистре. Уберем их при помощи функции `str.lower()` и проверим, ушли ли эти дубликаты: 

In [36]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверим столбцы **family_status** и **income_type** на уникальность значений. Видим, что дубликатов нет, поэтому можем переходить к изучению последнего столбца **purpose**:

In [37]:
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [38]:
data['income_type'].value_counts()

сотрудник          11050
компаньон           5054
пенсионер           3839
госслужащий         1453
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [39]:
data['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

В столбце **purpose** присутствует немало дубликатов. Заменим данные в столбце на унифицированные категории: 
* свадьба, 
* автомобиль, 
* образование, 
* жилая недвижимость, 
* коммерческая недвижимость, 
* недвижимость (для запросов на недвижимость без указания жилая/нежилая). 

Напишем функцию, которая заменит данные на корректные и создаст новый столбец **purpose_new**:

In [40]:
def new_purpose(purpose_row):
    if 'свадьб' in purpose_row:
        return 'свадьба'
    if 'жил' in purpose_row:
        return 'жилая недвижимость'
    if 'авт' in purpose_row:
        return 'автомобиль'
    if 'коммер' in purpose_row:
        return 'коммерческая недвижимость'
    if 'образ' in purpose_row:
        return 'образование'
    if 'недв' in purpose_row:
        return 'недвижимость'
        
data['new_purpose'] = data['purpose'].apply(new_purpose)
data['new_purpose'].value_counts()

жилая недвижимость           5675
автомобиль                   4288
образование                  3997
недвижимость                 3798
свадьба                      2337
коммерческая недвижимость    1307
Name: new_purpose, dtype: int64

После преобразования при помощи использования функции `new_purpose` создался столбец **new_purpose** с преобразованными целями, без дубликатов. Удалим исходный столбец **purpose** и для удобства переименуем **new_purpose** обратно в **purpose**:

In [41]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,18250,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [42]:
data = data.drop('purpose', axis = 1)
data = data.rename(columns = {'new_purpose': 'purpose'})
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилая недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилая недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,18250,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


Таким образом, были проверены столбцы с категорийными переменными на наличие дубликатов, и удалены дубликаты из столбцов **education** и **purpose**. Для удаления дубликатов из столбца **education** было достаточно метода приведения к нижнему регистру `str.lower()` - дубликаты могли возникнуть по невнимательности в процессе занесения данных (данные были занесены в разном регистре). Для удаления дубликатов из столбца purpose была создана функция `new_purpose`, которая отнесла каждую из целей к одной из 6 подходящих категорий. В будущем при занесении данных я бы рекомендовала создать список приемлимых категорий и заносить данные только исходя из этого списка (т.е. покупка квартиры, расширение жилплощади и проч. - отнести в жилую недвижимость). 

После преобразования датафрейма, приведения данных к единому формату записи и удаления неявных дубликатов, обратимся к явным дубликатам. Проверим, остались ли явные дубликаты и если да- удалим их и обновим индексы таблицы. 

In [44]:
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

280


In [45]:
data.duplicated().sum()

0

Как видно, явные дубликаты также были удалены из таблицы. Теперь можно приступать к следующему пункту

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Для упрощения исходного датафрейма создадим два датафрейма-словаря, в которых будем хранить значения **education**: **education_id** и **family_status : family_status_id**. Затем удалим столбцы **family_status** и **education**, оставив и исходном датафрейме только ключи- **education_id** и **family_status_id**:

In [46]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [47]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [48]:
data = data.drop(['education', 'family_status'], axis = 1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,жилая недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,жилая недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,18250,53,1,1,F,пенсионер,0,158616,свадьба


### Категоризация дохода.

Для упрощения анализа разделим некоторые данные на категории. Создадим функцию `category`, которая присвоит заемщику определенную категорию в зависимости от суммы заявленного дохода. Данные о рейтинге поместим в новый столбец **total_income_category**. 

In [49]:
def category(total):
    if 0 <= total <= 30000:
        return 'E'
    if 30001 <= total <= 50000:
        return 'D'
    if 50001 <= total <= 200000:
        return 'C'
    if 200001 <= total <= 1000000:
        return 'B'
    if 1000001 <= total:
        return 'A'

data['total_income_category'] = data['total_income'].apply(category)

In [50]:
data['total_income_category'].value_counts()

C    15713
B     5013
D      349
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

Аналогично с предыдущим шагом, создадим функцию `purpose_category`, которая разделит цели кредита на 4 категории. 

In [51]:
def purpose_category(purpose):
    if purpose == 'автомобиль':
        return 'операции с автомобилем'
    if purpose == 'свадьба':
        return 'проведение свадьбы'
    if purpose == 'образование':
        return 'получение образования'
    
    return 'операции с недвижимостью'
        
data['purpose_category'] = data['purpose'].apply(purpose_category)

In [52]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,жилая недвижимость,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,жилая недвижимость,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,B,получение образования
4,0,18250,53,1,1,F,пенсионер,0,158616,свадьба,C,проведение свадьбы


## Вопросы исследования

### Нахождение зависимости между количеством детей и возвратом кредита в срок

Для нахождения зависимости между количеством детей и возвратом кредита в срок сделаем следующее: 
* Найдем среднее значение просрочки по кредитам - `mean_debt`  <a id='intro'></a>,
* Найдем среднее значение просрочки по кредитам в зависимости от наличия детей у заемщика - `mean_child`,
* Создадим сводную таблицу, в которой строками будет количество детей у заемщика(**children**), а столбцами- - цели заема (с указанием исходных подкатегорий - **purpose_category**, **purpose**). Значениями таблицы возьмем среднее значение просрочки по кредитам (**debt**). 

In [53]:
mean_debt = data['debt'].mean() 
mean_debt

0.08195246662247893

In [54]:
mean_child = data.groupby('children')['debt'].mean()
mean_child 

children
0    0.076315
1    0.092848
2    0.094866
3    0.082067
4    0.097561
5    0.000000
Name: debt, dtype: float64

In [55]:
data_child = data.pivot_table(index = ['purpose_category', 'purpose'], columns = 'children', values = 'debt', aggfunc = 'mean')
data_child

children                                                   0         1  \
purpose_category         purpose                                         
операции с автомобилем   автомобиль                 0.085968  0.107292   
операции с недвижимостью жилая недвижимость         0.068796  0.069423   
                         коммерческая недвижимость  0.075472  0.085246   
                         недвижимость               0.064673  0.100592   
получение образования    образование                0.088009  0.104651   
проведение свадьбы       свадьба                    0.075958  0.096226   

children                                                   2         3  \
purpose_category         purpose                                         
операции с автомобилем   автомобиль                 0.120603  0.083333   
операции с недвижимостью жилая недвижимость         0.084112  0.086957   
                         коммерческая недвижимость  0.055118  0.047619   
                         недвижимость               0.097297  0.071429   
получение образования    образование                0.114713  0.058824   
проведение свадьбы       свадьба                    0.056075  0.156250   

children                                                   4    5  
purpose_category         purpose                                   
операции с автомобилем   автомобиль                 0.100000  0.0  
операции с недвижимостью жилая недвижимость         0.214286  0.0  
                         коммерческая недвижимость  0.000000  NaN  
                         недвижимость               0.000000  0.0  
получение образования    образование                0.000000  0.0  
проведение свадьбы       свадьба                    0.000000  0.0

## При рассмотрении сводной таблицы видим, что:

* У заемщиков, имеющих 5 детей, отсутсвуют просрочки по кредитам. Эти данные мы проверим также, подсчитав количество уникальных значений в столбце debt у заемщиков с 5 детьми (ниже). Можно считать заемщиков с пятью детьми самыми дисциплинированными, однако заемщики с таким количеством детей являются редкостью (9 человек на 21402 исследуемых), так что считать это закономерностью не стоит. 
* Больше всего просрочек по кредитам имеют заемщики с 4 детьми (`0.098` [среднего значения](#intro) просрочки против `0.082` просрочки в целом по таблице). Также в этом столбце наблюдается максимальное значение просрочки по кредиту среди всех значений - `0.214` при операциях с жилой недвижимостью. Это значит, что каждый пятый заемщик, имеющий 4х детей и запрашивающи кредит на жилую недвижимость, имеет просрочку. 
* Заемщики с 1 и 2 детьми имеют значение просрочки выше среднего (`0.092` и `0.094` / `0.082`). Чаще всего просрочка встречается при кредитах на автомобиль (1 - `0.107`, 2 - `0.120`) и кредитах на образование (1 - `0.104`, 2- `0.114`)
* Заемщики с 3 детьми имеют процент просрочки, близкий к среднему (`0.082 / 0.082`). Максимальный процент просрочки эта категория имеет при получении кредита на свадьбу (`0.156`) и операциях с жилой недвижимостью (`0.083`)
* Бездетные заемщики имеют процент просрочки чуть ниже среднего (`0.076 / 0.082`), больше всего процент просрочки при кредитах на образование и покупку автомобиля (`0.088` и `0.085`)

Тем самым видно, что наиболее дисциплинированными заемщиками являются заемщики без детей, с 3 и 5 детьми, заемщики с 1 и 2 детьми имеют процент просрочки выше среднего.

In [56]:
(data.loc[data['children'] == 5, 'debt']).value_counts()

0    9
Name: debt, dtype: int64

### Нахождение зависимости между семейным положением и возвратом кредита в срок

Для исследования,влияет ли семейное положение на возврат кредита в срок, найдем среднее значение просрочки в зависимости от семейного статуса. 
Для начала нам необходимо соединить исходный датафрейм **data** c датафреймом-словарем **family_status_dict** при помощи функции `merge()`, а затем найти среднее значение просрочки по каждому семейному статусу. 

In [57]:
data_sub_family = data.merge(family_status_dict, on = 'family_status_id', how = 'left')

In [58]:
mean_family = data_sub_family.groupby('family_status')['debt'].mean()
mean_family 

family_status
Не женат / не замужем    0.098096
в разводе                0.070707
вдовец / вдова           0.067021
гражданский брак         0.093628
женат / замужем          0.076535
Name: debt, dtype: float64

Видно, что процент просрочки меньше всего у вдовцов (`0.067 / 0.082` [среднего значения](#intro)), далее идут заемщики в разводе (`0.070 / 0.082`) и женатые заемщики (`0.076 / 0.082`). Набольшее значение просрочки у свободных заемщиков (`0.098 / 0.082`) и у состоящих в гражданском браке (`0.093 / 0.082`). 

Если рассмотреть сводную таблицу зависимости семейного статуса от целей кредитования, то наиболее часто просрачивают люди, находящиеся в гражданском браке и получающие кредит на образование (`0.149`), далее идут операции с автомобилем у неженатых (`0.129`) и у состоящих в гражданском браке (`0.119`). 
Меньше всего имеют просрочку вдовцы, проводящие операции с недвижимостью (`0.052`)

In [59]:
data_family = data_sub_family.pivot_table(index = 'family_status', columns = 'purpose_category', values = 'debt', aggfunc = 'mean')
data_family

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
family_status,,,,
Не женат / не замужем,0.129952,0.081646,0.108392,NaN
в разводе,0.075269,0.068452,0.071730,NaN
вдовец / вдова,0.092593,0.052930,0.076923,NaN
гражданский брак,0.119159,0.092105,0.149254,0.079773
женат / замужем,0.083612,0.070691,0.084880,NaN


### Нахождение зависимости между уровнем дохода и возвратом кредита в срок

Изучим зависимость просрочек кредита от заявленного дохода заемщика при помощи нахождения среднего значения просрочки по каждой категории и составления сводной таблицы по аналогии с вышеуказанными. 

In [60]:
mean_total = data_sub_family.groupby('total_income_category')['debt'].mean()
mean_total 

total_income_category
A    0.080000
B    0.070616
C    0.086043
D    0.060172
E    0.090909
Name: debt, dtype: float64

Видно, что меньше всего доля просрочки у клиентов из категории **D** (доход от 30001 до 50000 руб.) - `0.060 / 0.082` [среднего значения](#intro), далее идут заемщики категории **В** (от 200001 до 1000000 руб.) - `0.070 / 0.082`. Следом идут близкие к среднему значению категории **А** (от 1000001 руб. и выше) и **С** (от 50001 до 200000 руб.) - `0.080 / 0.082` и `0.086 / 0.082`. Больше всех долю просрочки имеет категория **Е** (доход до 30000 руб.) - `0.090 / 0.081`

In [61]:
data_total = data.pivot_table(index = 'total_income_category', columns = 'purpose_category', values = 'debt', aggfunc = 'mean')
data_total

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
total_income_category,,,,
A,0.000000,0.058824,0.250000,0.000000
B,0.083170,0.067266,0.077528,0.051471
C,0.096600,0.075909,0.100169,0.088889
D,0.138889,0.035714,0.026667,0.088235
E,0.000000,0.181818,0.000000,0.000000


Если мы изучим сводную таблицу, то увидим, что просрочка отсутствует: 
* Категория **А** - операции  автомобилем и проведение свадьбы;
* Категория **Е** - операции с автомобилем, получение образования и проведение свадьбы.

Парадоксально, но самый высокий процент просрочки по категориям мы видим у самой высокооплачиваемой категории **А** с целью кредита "получение образования" - здесь просрочка у каждого четвертого заемщика `(0.250)`. Далее идут "операции с недвижимостью" у категории **Е** `(0.181)` и замыкает тройку "операции с автомобилем" категории **D** `(0.138)`

### Как разные цели кредита влияют на его возврат в срок

В пунктах выше была рассмотрена зависимость целей кредита от его возврата в срок. Я считаю, что данную зависимость следует рассматривать в комплексе с другими параметрами, такими как количество детей, семейное положение, доход, образование и проч. В каждом из указанных пунктов мною была рассмотрена зависимость целей кредита от его возврата в срок. Однако мы также можем посчитать среднее число просрочки кредита в зависимости от цели кредитования (по категориями **purpose_category**).

In [62]:
mean_purpose = data.groupby('purpose_category')['debt'].mean()
mean_purpose

purpose_category
операции с автомобилем      0.093993
операции с недвижимостью    0.073281
получение образования       0.093679
проведение свадьбы          0.079773
Name: debt, dtype: float64

Если сравнить долю просрочки по категориям со средним значением, видно, что меньше всего просрачивают кредит на операции с недвижимостью (`0.073 / 0.082`) и проведение свадьбы (`0.079 / 0.082`) [среднего значения](#intro). Выше среднего- получение образования (`0.093 / 0.082`) и операции с автомобилем (`0.093 / 0.082`)

При использовании сводной таблицы можно изучить зависимость просрочки кредита от целей, семейного статуса и наличия детей.
Из этой таблицы можно заметить, что самая высокая доля просрочки кредита (`0.500`, т.е. каждый второй заемщик!) наблюдается при получении кредита на операцию с недвижимостью у неженатого заемщика с 3 детьми. 
Следом идут операции с автомобилем у опять же неженатого заемщика с 2 детьми (`0.250`) и получение образования у заемщика, находящегося в гражданском браке и имеющего 1 ребенка (`0.187`). 

In [63]:
data_family_child = data_sub_family.pivot_table(index = ['children', 'family_status'], columns = 'purpose_category', values = 'debt', aggfunc = 'mean')
data_family_child

purpose_category                операции с автомобилем  \
children family_status                                   
0        Не женат / не замужем                0.124514   
         в разводе                            0.066667   
         вдовец / вдова                       0.095477   
         гражданский брак                     0.100000   
         женат / замужем                      0.072526   
1        Не женат / не замужем                0.139785   
         в разводе                            0.088235   
         вдовец / вдова                       0.000000   
         гражданский брак                     0.160377   
         женат / замужем                      0.098240   
2        Не женат / не замужем                0.250000   
         в разводе                            0.142857   
         вдовец / вдова                       0.166667   
         гражданский брак                     0.114286   
         женат / замужем                      0.111455   
3        Не женат / не замужем                0.000000   
         в разводе                            0.000000   
         вдовец / вдова                            NaN   
         гражданский брак                     0.400000   
         женат / замужем                      0.060000   
4        Не женат / не замужем                     NaN   
         в разводе                            0.000000   
         вдовец / вдова                            NaN   
         гражданский брак                     0.000000   
         женат / замужем                      0.125000   
5        гражданский брак                     0.000000   
         женат / замужем                      0.000000   

purpose_category                операции с недвижимостью  \
children family_status                                     
0        Не женат / не замужем                  0.074161   
         в разводе                              0.067285   
         вдовец / вдова                         0.047414   
         гражданский брак                       0.078582   
         женат / замужем                        0.067067   
1        Не женат / не замужем                  0.113725   
         в разводе                              0.071038   
         вдовец / вдова                         0.081633   
         гражданский брак                       0.122605   
         женат / замужем                        0.072447   
2        Не женат / не замужем                  0.073171   
         в разводе                              0.062500   
         вдовец / вдова                         0.166667   
         гражданский брак                       0.131148   
         женат / замужем                        0.082759   
3        Не женат / не замужем                  0.500000   
         в разводе                              0.100000   
         вдовец / вдова                         0.000000   
         гражданский брак                       0.000000   
         женат / замужем                        0.078014   
4        Не женат / не замужем                  1.000000   
         в разводе                                   NaN   
         вдовец / вдова                         0.000000   
         гражданский брак                       0.000000   
         женат / замужем                        0.133333   
5        гражданский брак                            NaN   
         женат / замужем                        0.000000   

purpose_category                получение образования  проведение свадьбы  
children family_status                                                     
0        Не женат / не замужем               0.112335                 NaN  
         в разводе                           0.082803                 NaN  
         вдовец / вдова                      0.069364                 NaN  
         гражданский брак                    0.130597            0.075958  
         женат / замужем                     0.076129                 NaN  
1        Не женат / не замужем              

### Вывод

При изучении была найдена зависимость между возвратом кредита и параметрами заемщика (наличие детей, семейный статус, уровень дохода). Для подтверждения гипотезы были построены сводные таблицы, которые подтвердили наличие зависимостей. 

## Общий вывод:

В ходе проектной работы был изучен датафрейм с информацией о заемщиках для нахождения зависимости семейного положения и количества детей клиента на факт погашения кредита в срок. Для того, чтобы провести данное исследование, была произведена предобработка датафрейма на наличие пропусков, дубикатов и аномальных данных. Далее был изучен сам вопрос исследования и была найдена зависимость между семейным положением и количеством детей клиента на факт погашения кредита в срок. В среднем, самыми дисциплинированными заемщиками являются бездетные клиенты, если смотреть с т.з. семейного статуса - вдовцы. Самыми недисциплинированными и просрачивающими-  заемщики с 1, 2 и 4 детьми или неженатые клиенты. 

Была составлена сводная таблица зависимости наличия детей, семейного статуса клиента и целей заема от доли просрочки и найдены сочетания с самой большой просрочкой. 